In [1]:
%gui asyncio

In [1]:
import asyncio    
from ipylgbst import LegoBoostWidget,LedColor,Port
boost = LegoBoostWidget()
boost

LegoBoostWidget(device_info={'ports': {'A': {'action': '', 'angle': 0}, 'B': {'action': '', 'angle': 0}, 'AB':…

In [2]:

async def main(boost, out):
    def print(x):
        out.append_stdout(f"{x}\n")
        

    print("jeah")

    print("await connect")
    await boost.connect()
    print("await done")
    #out.append_stdout("connect done\n")


    # print("stuff")
    await boost.drive(3)
    
    # todo also have single function for that
    while True:
        import asyncio
        if await boost.get_distance() <80:
            break

    await boost.turn(360)
    await asyncio.sleep(2)

    await boost.drive(3)
    await asyncio.sleep(1)

    await boost.drive(-3)
    await boost.turn(180)
    
future = boost.run(main)

Output()

In [3]:

async def main(boost, out):
    def p(x):
        out.append_stdout(f"{x}\n")
        
    await boost.connect()
    
    await boost.motorTime(Port.A, 20,-10)
    while True:
        import asyncio
        p("a")
        await asyncio.sleep(1)
        p("b")
        d =  await boost.get_distance()
        p(d)
        if d < 80:
            await boost.motorTime(Port.A, 0,0)
            break
    
future = boost.run(main)


Output()